In [1]:
from keras import Sequential, Model
import numpy as np
from keras import Input
from keras import layers
from tensorflow.keras.optimizers import RMSprop

import sys
sys.path.append('..')

import ep.etools as apt


In [2]:
# import des données financières de Yahoo
import yfinance as yf
etf = ['BZ=F','EURUSD=X']
data = yf.Ticker(etf[0])

In [3]:
dataH = data.history(start='2007-01-01',period='1d')
# dataH

In [4]:
it = yf.download(tickers=etf, period='1d',start='2009-01-01')
it.describe()

[*********************100%***********************]  2 of 2 completed


Adj Close                     Close                      High  \
              BZ=F     EURUSD=X         BZ=F     EURUSD=X         BZ=F   
count  3320.000000  3493.000000  3320.000000  3493.000000  3320.000000   
mean     75.820801     1.229171    75.820801     1.229171    76.668379   
std      25.389233     0.116042    25.389233     0.116042    25.407977   
min      19.330000     1.037958    19.330000     1.037958    21.270000   
25%      55.342499     1.125404    55.342499     1.125404    56.067500   
50%      71.605000     1.203239    71.605000     1.203239    72.420002   
75%     103.365000     1.328745   103.365000     1.328745   104.265001   
max     127.980003     1.512791   127.980003     1.512791   137.000000   

                            Low                      Open               \
          EURUSD=X         BZ=F     EURUSD=X         BZ=F     EURUSD=X   
count  3493.000000  3320.000000  3493.000000  3320.000000  3493.000000   
mean      1.233548    74.898506     1.224598    75.799133     1.229201   
std       0.116759    25.321962     0.115607    25.365659     0.116129   
min       1.041938    16.000000     0.760572    19.559999     1.037958   
25%       1.129801    54.679999     1.122057    55.369999     1.125467   
50%       1.207001    70.670002     1.199904    71.580002     1.203254   
75%       1.334205   102.262501     1.323592   103.315002     1.328745   
max       1.514532   125.300003     1.504099   129.570007     1.512608   

             Volume           
               BZ=F EURUSD=X  
count   3320.000000   3493.0  
mean   17896.688855      0.0  
std    16685.746123      0.0  
min        0.000000      0.0  
25%      948.750000      0.0  
50%    16651.000000      0.0  
75%    30898.750000      0.0  
max    87185.000000      0.0

In [5]:
# supprime colonnes inutiles
it.drop(['High','Low','Open','Volume','Adj Close'],axis=1,inplace=True)
print(it.columns.to_list())
# it.rename(columns='_'.join,inplace=True)
it.columns = it.columns.map('_'.join)
it.rename(columns={'Close_BZ=F':'brent','Close_EURUSD=X':'FX'},inplace=True)
it.describe()

[('Close', 'BZ=F'), ('Close', 'EURUSD=X')]


,brent,FX
count,3320.000000,3493.000000
mean,75.820801,1.229171
std,25.389233,0.116042
min,19.330000,1.037958
25%,55.342499,1.125404
50%,71.605000,1.203239
75%,103.365000,1.328745
max,127.980003,1.512791


In [6]:
it.dropna(axis=0,inplace=True)
it.describe()

,brent,FX
count,3315.000000,3315.000000
mean,75.814537,1.227582
std,25.388360,0.115928
min,19.330000,1.037958
25%,55.334999,1.124891
50%,71.620003,1.195943
75%,103.305000,1.327616
max,127.980003,1.512791


In [7]:
import matplotlib.pyplot as plt
it_plot = it.copy()
# normalisation
it_mean = it.mean(axis=0)
it_std = it.std(axis=0)
it_plot = (it_plot - it_mean) /it_std
# plot
plt.figure(figsize=(18,10))
plt.xlabel('Date')
plt.ylabel('cours en $')
plt.plot(it_plot,label=it_plot.columns.to_list())
plt.legend(loc='upper left')
plt.show()

In [8]:
# parametres
scale            = 1        # % du dataset (1=all)
train_prop       = .8       # ration du train vs test
sequence_len     = 8
batch_size       = 32
epochs           = 10
features         = ['brent','FX']
features_len     = len(features)

In [9]:
# Mise à l'échelle du dataFrame

df = it[:int(scale*len(it))].reset_index()
train_len=int(train_prop*len(df))

print(df)

# ---- Train / Test
dataset_train = df.loc[ :train_len-1, features ]
dataset_test  = df.loc[train_len:,    features ]
apt.subtitle('Train dataset :')
display(dataset_train.head(15))

# ---- Normalize, and convert to numpy array

mean = dataset_train.mean()
std  = dataset_train.std()
dataset_train = (dataset_train - mean) / std
dataset_test  = (dataset_test  - mean) / std

apt.subtitle('Après normalisation :')
display(dataset_train.describe().style.format("{0:.2f}"))

dataset_train = dataset_train.to_numpy()
dataset_test  = dataset_test.to_numpy()

apt.subtitle('Shapes :')
print('Dataset       : ',df.shape)
print('Train dataset : ',dataset_train.shape)
print('Test  dataset : ',dataset_test.shape)

           Date       brent        FX
0    2008-12-31   45.590000  1.397995
1    2009-01-02   46.910000  1.392292
2    2009-01-05   49.619999  1.364592
3    2009-01-06   50.529999  1.351004
4    2009-01-07   45.860001  1.361100
...         ...         ...       ...
3310 2022-05-23  113.419998  1.056982
3311 2022-05-24  113.559998  1.068205
3312 2022-05-25  114.029999  1.073653
3313 2022-05-26  117.400002  1.068661
3314 2022-05-27  119.250000  1.073295

[3315 rows x 3 columns]


<br>**Train dataset :**

,brent,FX
0,45.590000,1.397995
1,46.910000,1.392292
2,49.619999,1.364592
3,50.529999,1.351004
4,45.860001,1.361100
5,44.669998,1.369901
6,44.419998,1.345497
7,42.910000,1.336005
8,44.830002,1.320202
9,45.080002,1.321702


<br>**Après normalisation :**

,brent,FX
count,2652.00,2652.00
mean,0.00,0.00
std,1.00,1.00
min,-2.00,-1.74
25%,-0.88,-0.97
50%,-0.17,0.06
75%,1.12,0.85
max,1.89,2.22


<br>**Shapes :**

Dataset       :  (3315, 3)
Train dataset :  (2652, 2)
Test  dataset :  (663, 2)


In [10]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [11]:
# ---- Train generator

train_generator = TimeseriesGenerator(dataset_train, dataset_train, length=sequence_len,  batch_size=batch_size)
test_generator  = TimeseriesGenerator(dataset_test,  dataset_test,  length=sequence_len,  batch_size=batch_size)

# ---- About

apt.subtitle('About the splitting of our dataset :')

x,y=train_generator[0]
print(f'Nombre de train batchs disponibles : ', len(train_generator))
print('batch x shape : ',x.shape)
print('batch y shape : ',y.shape)

x,y=train_generator[0]
apt.subtitle('What a batch looks like (x[0]) :')
apt.np_print(x[0] )
apt.subtitle('What a batch looks like (y[0]) :')
apt.np_print(y[0])

<br>**About the splitting of our dataset :**

Nombre de train batchs disponibles :  83
batch x shape :  (32, 8, 2)
batch y shape :  (32, 2)


<br>**What a batch looks like (x[0]) :**

[[-1.304  1.259]
 [-1.252  1.212]
 [-1.145  0.98 ]
 [-1.109  0.866]
 [-1.294  0.951]
 [-1.341  1.024]
 [-1.35   0.82 ]
 [-1.41   0.741]]


<br>**What a batch looks like (y[0]) :**

[-1.334  0.609]


In [12]:
# tf.config.get_visible_devices()
# tf.config.PhysicalDevice('CPU:0','CPU')

In [13]:
print(tf.__version__)

2.7.0


In [14]:
run_dir = './run/'
save_dir = f'{run_dir}/best_model.h5'
bestmodel_callback = tf.keras.callbacks.ModelCheckpoint(filepath=save_dir, verbose=0, save_best_only=True)

In [15]:
# le model Keras
input_data = Input(shape=(sequence_len,features_len))
x = layers.GRU(100, dropout=0.2,return_sequences=True, recurrent_dropout=0.5)(input_data)
# x = layers.LSTM(100, dropout=0.1, recurrent_dropout=0.5, return_sequences=True)(input_data)
# x = layers.GRU(32, dropout=0.2,recurrent_dropout=0.5, return_sequences=True)(x)
x = layers.GRU(32, dropout=0.2,recurrent_dropout=0.5,activation='relu')(x)
x = layers.Dense(32,activation='relu')(x)
# x = layers.Dropout(0.2)(x)
output_data = layers.Dense(features_len)(x)
# with tf.device("/cpu:0"):
model = Model(input_data,output_data)
    # model.compile(optimizer='rmsprop',loss='categorical_crossentropy',metrics=['mae'])
model.compile(optimizer=RMSprop(),loss='mse',metrics=['mae'])
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 8, 2)]            0         
                                                                 
 gru (GRU)                   (None, 8, 100)            31200     
                                                                 
 gru_1 (GRU)                 (None, 32)                12864     
                                                                 
 dense (Dense)               (None, 32)                1056      
                                                                 
 dense_1 (Dense)             (None, 2)                 66        
                                                                 
Total params: 45,186
Trainable params: 45,186
Non-trainable params: 0
_________________________________________________________________


In [16]:
# input_data = Input(shape=(sequence_len,features_len))
# x1 = layers.LSTM(100, activation='relu')(input_data)
# x2 = layers.Conv1D()(input_data)
# x2 = layers.MaxPooling1D()(x2)
# x3 = layers.Conv1D()(x2)
# x3 = layers.MaxPooling1D()(x3)


In [17]:
# with tf.device("/device:cpu:0"):
history = model.fit(train_generator,epochs=epochs,validation_data=test_generator,verbose=1,use_multiprocessing=True,callbacks = [bestmodel_callback])

2022-05-29 20:09:47.441115: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


Epoch 1/10
 1/83 [..............................] - ETA: 3:44 - loss: 1.1622 - mae: 1.0589

: 

: 

In [ ]:
apt.plot_history(history,plot={'loss':['loss','val_loss'], 'mae':['mae','val_mae']}, save_as='01-history')

In [ ]:
best_model = tf.keras.models.load_model(f'{run_dir}/best_model.h5')

In [ ]:
import math,random
def denormalize(mean,std,seq):
    nseq = seq.copy()
    for i,s in enumerate(nseq):
        s = s*std + mean
        nseq[i]=s
    return nseq


# ---- Get a sequence

i=random.randint(0,len(dataset_test)-sequence_len)
j = len(dataset_test)-sequence_len
sequence      = dataset_test[i:i+sequence_len]
sequence_true = dataset_test[i:i+sequence_len+1]
print(sequence_len,i,j,len(dataset_train))

# ---- Prediction

pred = best_model.predict( np.array([sequence]) )

# ---- De-normalization

sequence_true = denormalize(mean,std, sequence_true)
pred          = denormalize(mean,std, pred)

print(pred)
# ---- Show it
feat=1

apt.plot_multivariate_serie(sequence_true, predictions=pred, labels=features, only_features=[feat],width=14, height=8)

delta=abs(sequence_true[-1][feat]-pred[-1][feat])
print(f'Gap between prediction and reality : {delta:.2f} u')

In [ ]:
feat=0

apt.plot_multivariate_serie(sequence_true, predictions=pred, labels=features, only_features=[feat],width=14, height=8)

delta=abs(sequence_true[-1][feat]-pred[-1][feat])
print(f'Gap between prediction and reality : {delta:.2f} u')

In [ ]:
def get_prediction(dataset, model, iterations=4,sequence_len=16):

    # ---- Initial sequence

    s=random.randint(0,len(dataset)-sequence_len-iterations)

    sequence_pred = dataset[s:s+sequence_len].copy()
    sequence_true = dataset[s:s+sequence_len+iterations].copy()

    # ---- Iterate

    sequence_pred=list(sequence_pred)

    for i in range(iterations):
        sequence=sequence_pred[-sequence_len:]
        pred = model.predict( np.array([sequence]) )
        sequence_pred.append(pred[0])

    # ---- Extract the predictions    

    pred=np.array(sequence_pred[-iterations:])

    # ---- De-normalization

    sequence_true = denormalize(mean,std, sequence_true)
    pred          = denormalize(mean,std, pred)

    return sequence_true,pred

In [ ]:
sequence_true, pred = get_prediction(dataset_test, best_model,iterations=5,sequence_len=40)

feat=[0,1]

apt.plot_multivariate_serie(sequence_true, predictions=pred, labels=features,
                            only_features=feat,width=14, height=8)